## <i>In this project, we will set up a traditional GWAS algorithm and expand on it by performing gene-set level association testing with a certain trait</i>

#### Set working diretory and import needed libraries

In [ ]:
%cd "C:\\Rami\\Career_Experience\\Andreas_Henschel\\Project\\Material_used"

In [ ]:
import hail as hl
import pandas as pd
from hail.plot import show, output_notebook
import ipywidgets as widgets
from biomart import BiomartServer
import io
import re
import subprocess
import matplotlib.pyplot as plt
import random
import ast
import plotly.io as pio
import plotly.express as px
pio.renderers.default = 'jupyterlab'

#### Using hail to prepare the data for GWAS

In [ ]:
#Initialize hail and set up env for interactive plot visualization (bokeh)
hl.init()
output_notebook()

In [ ]:
#import the vcf file of SNP data and write the raw data into a matrix table (not real data only for practice)
vcf = hl.import_vcf("BroadE_HailWorkshop2020/resources/1kg.vcf.bgz", min_partitions=4)
vcf.write("1kg.mt", overwrite=True)
mt = hl.read_matrix_table("1kg.mt")

In [ ]:
mt.describe()

In [ ]:
#add a row field to display the chromosome number of every variant
mt = mt.annotate_rows(contig=mt.locus.contig)

#### Quality control of raw data

In [ ]:
#compute various quality control metrics for every column (individual/sample)
mt = hl.sample_qc(mt)

#keep the columns (samples) who have a proportion of non-missing genotypes >= 95%
mt = mt.filter_cols(mt.sample_qc.call_rate >= 0.95)

#compute various QC metrics for every row/variant and adds a new row field with these metrics
mt = hl.variant_qc(mt)
mt = mt.filter_rows(mt.variant_qc.call_rate > 0.95)

#keep variants that have a variance > specified threshold (0.001). The variance here is for the number of alternate alleles every variant has.
# An individual can have a genotype of 0 (homozygous reference), 1 (heterozygous), or 2 (homo alternates) for a specific variant. 
threshold = 0.001
mt = mt.filter_rows(hl.agg.stats(mt.GT.n_alt_alleles()).stdev ** 2 > threshold)

#keeps variants that have alt allele frequency > 1% for every variant across all samples
mt = mt.filter_rows(hl.agg.call_stats(mt.GT, mt.alleles).AF[1] > 0.01)

#### We now handle and process the phenotype and covariate files

In [ ]:
#import the phenotype file
table = hl.import_table("C:\\Rami\\Career_Experience\\Andreas_Henschel\\Project\\Material_used\\BroadE_HailWorkshop2020\\resources\\phenotype_file\\1kg_annotations.txt", impute=True, key='s')

#filter out the table so that only the samples found in the mt are kept in the table
table = table.filter(hl.is_defined(mt.cols()[table.key]))

#add column fields using the imported table
mt = mt.annotate_cols(
    fam_id=mt.s,
    ind_id=mt.s,
    pat_id="NA",
    mat_id="NA",
    female=table[mt.s].is_female,
    caff=table[mt.s].caffeine_consumption
)

filtered_table = mt.cols().select("ind_id", "fam_id", "pat_id", "mat_id", "female", "caff")
filtered_table.export("C:\\Rami\\Career_Experience\\Andreas_Henschel\\Project\\Material_used\\BroadE_HailWorkshop2020\\resources\\phenotype_file\filtered_1kg_annotations_test.txt")

#### Create PLINK genotype files from the matrix table

In [ ]:
hl.export_plink(mt, 'C:\\Rami\\Career_Experience\\Andreas_Henschel\\Project\\Material_used\\exported_Gene_Data\\PLINK_genotypes_from_hail\\test', 
        fam_id=mt.s, ind_id=mt.s, pat_id=None, mat_id=None, is_female=mt.female, pheno=mt.caff)

#### Preprocessing phenotypes and covariates for Regenie (tool to perform GWAS with enhanced parameters)

In [ ]:
#modify 'table' for regenie format:

#read the table of phenotypes with tab as delimiter
pheno = pd.read_csv("BroadE_HailWorkshop2020/resources/filtered_1kg_annotations.txt", sep="\t")

#Rename the columns to add FamilyID(FID) and IndividualID(IID) for gwas study
pheno.columns = ['s', 'IID', 'FID', 'pat_id', 'mat_id', 'is_female', 'caffeine_consumption']

#Create a new dataframe via [[]] using subset of columns from original dataframe 
pheno = pheno[['FID', 'IID', 'caffeine_consumption']]

pheno.to_csv('exported_Gene_Data/phenotypes/filtered_pheno.txt', index=False, sep='\t')

In [ ]:
cov = pd.read_csv("BroadE_HailWorkshop2020/resources/filtered_1kg_annotations.txt", sep="\t")

cov.columns = ['s', 'IID', 'FID', 'pat_id', 'mat_id', 'is_female', 'caffeine_consumption']
cov['FID'] = cov['IID']   #copy column IID to new column FID

cov = cov[['FID', 'IID', 'is_female']]
#cov.to_csv("C:\\Rami\\VScode\\hail tool\\exported_data\\phenotypes\\cov.txt", index=False, sep='\t')

for index,row in cov.iterrows():
    if row['is_female']==True:
        cov.at[index, 'is_female']=1
    elif row['is_female']==False:
        cov.at[index, 'is_female']=0

cov.to_csv('exported_Gene_Data/phenotypes/filtered_cov.txt', index=False, sep='\t')

#### Performing the GWAS using REGENIE

In [ ]:
%%bash
# Runs the first step of regenie, which fits a ridge regression model to predict phenotypes based on covariates and genotype principal components.
regenie \
--step 1 \
--bed "exported_Gene_Data/PLINK_genotypes/geno" \
--covarFile "exported_Gene_Data/phenotypes/filtered_cov.txt" \
--catCovarList "is_female" \
--phenoFile "exported_Gene_Data/phenotypes/filtered_pheno.txt" \
--bsize 200 \
--out "regenie_output/step1"

In [ ]:
%%bash
# Runs the second step of regenie, where it tests each genetic variant for association with the phenotype(s) while controlling for confounding effects.
regenie \
--step 2 \
--bed "exported_Gene_Data/PLINK_genotypes/geno" \
--phenoFile "exported_Gene_Data/phenotypes/filtered_pheno.txt" \
--bsize 200 \
--pred "regenie_output/step1_pred.list" \
--out "regenie_output/step2"

#### Post modifications of GWAS output file to make the non-numerical CME match the SNP ID ('CME' : 'gene_pos')

In [ ]:
with open('regenie_output/step2_caffeine_consumption.regenie', 'r') as file:
    content = file.read()
pattern = r'\b23\b'
updated_regenie = re.sub(pattern, 'X' , content)

with open('regenie_output/step2_caffeine_consumption_updated.regenie', 'w') as file:
    file.write(updated_regenie)

#### Creating a CSV file of the GWAS results from regenie

In [ ]:
#initialize a new matrix table from the output file of regenie
regenie_table = hl.import_table('regenie_output/step2_caffeine_consumption_updated.regenie', delimiter=' ')

regenie_table = regenie_table.annotate(locus=hl.locus(regenie_table.CHROM, hl.int(regenie_table.GENPOS)))
regenie_table = regenie_table.annotate(LOG10P=hl.float64(regenie_table.LOG10P))
regenie_table = regenie_table.annotate(p_values=hl.float64(10**-regenie_table.LOG10P))

regenie_table.export("regenie_output/GWAS_results.txt")

## <i>Now it's time to map these raw SNPs to their genes based on their genomic position

#### Retrieve Homo Sapien gene data from Biomart

In [ ]:
server = BiomartServer("http://www.ensembl.org/biomart")
dataset = server.datasets['hsapiens_gene_ensembl']

# Define the attributes to retrieve
attributes = ['chromosome_name', 'start_position', 'end_position', 'external_gene_name', 'ensembl_gene_id', 'entrezgene_id']

# Perform the query and retrieve the results
result = dataset.search({
    'attributes': attributes
})

gene_data = result.text

gene_df = pd.read_csv(io.StringIO(gene_data), delimiter="\t", names=attributes)
gene_df['entrezgene_id'] = gene_df['entrezgene_id'].astype('Int64')  # Use 'Int64' for nullable integers

gene_df.to_csv('HomoSapien_genes/HS_genes.txt', sep='\t', index=False, header=False)

#### Function to map non-numeral CMEs to integers for initial sorting

In [ ]:
def custom_sort_key(val):
    pattern = r'\b\d\d?\b'
    
    if re.match(pattern, val):
        return int(val)
    else:
        chrom_order = {'X': 23, 'Y': 24, "MT":25}
        return chrom_order.get(val, 26)

df = pd.read_csv("HomoSapien_genes/HS_genes.txt", sep='\t')
df.columns=["chrom", "start", "end", "gene_name", "Ensembl_gene_id", "NCBI_gene_ID"]
df_sorted = df.sort_values(by=['chrom', 'start'], key=lambda x: x.map(custom_sort_key) if x.name=='chrom' else x)
df_sorted.to_csv('HomoSapien_genes/HS_initial_sorted_genes.txt', sep='\t', index=False, header=False)

#### Enhance the sorting process of the genes file retrieved from Biomart by separating numerical CME from non-numerical ones

In [ ]:
# Steps for indexing the genes file since running a direct sort on the file does not do the trick 
%%bash
total=$(cat "HomoSapien_genes/HS_initial_sorted_genes.txt" | wc -l)
last_CME_line_with_MT=$(cat "HomoSapien_genes/HS_initial_sorted_genes.txt" | grep -n "^MT" | tail -1 | cut -d ":" -f1)
diff=$((total-last_CME_line_with_MT))

head -$last_CME_line_with_MT "HomoSapien_genes/HS_initial_sorted_genes.txt" > "HomoSapien_genes/HS_sorted_CME.txt"
tail -$diff "HomoSapien_genes/HS_initial_sorted_genes.txt" > "HomoSapien_genes/HS_unsorted_haplotypes.txt"
sort -k1,1 -k2,2n "HomoSapien_genes/HS_unsorted_haplotypes.txt" > "HomoSapien_genes/HS_sorted_haplotypes.txt"
cat HS_sorted_CME.txt "HomoSapien_genes/HS_sorted_haplotypes.txt" > "HomoSapien_genes/HS_final_sorted_genes.txt"

# indexing the sorted file to easily extract data from it
bgzip "HomoSapien_genes/HS_final_sorted_genes_copy.txt"
tabix -p bed "HomoSapien_genes/HS_final_sorted_genes_copy.txt.gz"

#### Script to retrieve gene data from raw SNP input: CME and SNP position (allows for discovery of new SNPs that are not recorded in the database)

In [ ]:
%%writefile "C:\\Rami\\Career_Experience\\Andreas_Henschel\\Project\\Material_used\\scripts\\gene_script.py"
import pysam
import argparse

def find_genes_with_snp(tabix_file, chrom, pos):
    if chrom == 23:
        chrom = 'X'
    region = f"{chrom}:{pos-1}-{pos}"     #pos-1 and pos to narrow down the search as much as possible
    genes_containing_snp = []
    
    for entry in tabix_file.fetch(region=region):     #it will retrieve all entries/genes that overlap with the specified region
        fields = entry.split('\t')
        gene_chrom = fields[0]
        gene_start = int(fields[1])
        gene_end = int(fields[2])
        gene_name = fields[3]
        gene_NCBI_id = fields[5]
        genes_containing_snp.append((gene_name, gene_NCBI_id, gene_start, gene_end))
    if len(genes_containing_snp) == 0:
        return "None"
    if len(genes_containing_snp) == 1:
        return genes_containing_snp[0]
    else:    #case of nested genes
        gene_of_interest = genes_containing_snp[0]
        min_start = gene_of_interest[2]
        min_end = gene_of_interest[3]
        for gene in genes_containing_snp:
            if gene[2] < min_start and gene[3] < min_end:
                gene_of_interest = gene
                min_start = gene[2]
                min_end = gene[3]
        return gene_of_interest
    
compressed_file="HomoSapien_genes/HS_final_sorted_genes_copy.txt.gz"
tabix_file = pysam.TabixFile(compressed_file)

if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Find genes containing SNP.')
    parser.add_argument('--snp_chrom', type=int, required=True, help='Chromosome of the SNP')
    parser.add_argument('--snp_pos', type=int, required=True, help='Position of the SNP')

    args = parser.parse_args()

    genes = find_genes_with_snp(tabix_file, args.snp_chrom, args.snp_pos)
    print(genes)

#### vcf_script file used to update the GWAS dataframe with genes of known SNPs from ncbi local database of SNPs

In [ ]:
%%writefile "C:\\Rami\\Career_Experience\\Andreas_Henschel\\Project\\Material_used\\scripts\\vcf_script.py"
#Script to create a GWAS dataframe with all data I retrieved for the raw SNPs
import pysam
import pandas as pd

def find_snp(vcf_file):                                    
    for i in range(len(GWAS_df)):                              
        chrom = GWAS_df['CHROM'][i]                            
        SNP_pos = GWAS_df['GENPOS'][i]
        records = vcf_file.fetch(chrom, SNP_pos-1, SNP_pos)
        #example of record:
        #CHROM     POS        ID      REF   ALT   QUAL  FILTER   INFO
        #1        55550    rs1234567   G     A     .      .     'dict object'

        for record in records:   #loop needed because 'records' is an iterator object (even if only one match)
            record_info = dict(record.info)  #example entry{'RS': 775809821, 'RSPOS': 10020, 'dbSNPBuildID': 144, 'SSR': 0, 'SAO': 0, 'VP': '0x050000020005000002000200', 'GENEINFO': 'DDX11L1:100287102', 'WGT': 1, 'VC': 'DIV', 'R5': True, 'ASP': True}
            if 'GENEINFO' in record_info:
                gene_info = record_info['GENEINFO'].split(":")
                GWAS_df.loc[(GWAS_df["CHROM"] == record.chrom) & (GWAS_df['GENPOS'] == record.pos), "Gene_name"] = gene_info[0]
                GWAS_df.loc[(GWAS_df["CHROM"] == record.chrom) & (GWAS_df['GENPOS'] == record.pos), "NCBI_ID"] = gene_info[1]
            GWAS_df.loc[(GWAS_df["CHROM"] == record.chrom) & (GWAS_df['GENPOS'] == record.pos), "ID"] = record.id

def search_gene(CHROM, SNP):
    # Whether the Gene name is known or not, we still need to go back to the 'genes file' to retrieve the start and end positions of this gene
    command = f'wsl python3 scripts/gene_script.py --snp_chrom {CHROM} --snp_pos {SNP}'
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    # Check the output
    out = result.stdout.strip()
    if out == "None":
        return "None"
    else:
        try:
            result_list = ast.literal_eval(out)  # Safely evaluate the string to a list
        except (SyntaxError, ValueError) as e:
            print(f"Error parsing result_str: {result_str}")
        return result_list


GWAS_df = pd.read_csv("regenie_output/GWAS_results.txt", sep='\t')

GWAS_df["Gene_name"] = 'NA'  #Use this column for when you find the Gene of the SNP; replace NA with gene name.
GWAS_df['NCBI_ID'] = 'NA'
GWAS_df["GENPOS"] = GWAS_df["GENPOS"].astype('int64')

GWAS_df["CHROM"] = GWAS_df["CHROM"].replace({"X":"23"})
GWAS_df["CHROM"] = GWAS_df["CHROM"].astype('int64')

GWAS_df["color"] = GWAS_df['CHROM'].apply(lambda x : 'blue' if x%2==0 else 'black')   #For coloring purposes of manhattan plot
GWAS_df['def_color'] = GWAS_df['color']


compressed_file="local_SNP_database/00-All.vcf.gz"
vcf_file = pysam.VariantFile(compressed_file)
find_snp(vcf_file)

#Some SNPs were not caught identified using the find_SNP function but they have data from the manually sorted HS_final_sorted_genes_copy.txt.gz file
#So, I am going to loop over the entire GWAS_df ONCE, find the gene_names of the unidentified SNPs using the search_gene() function.
for index, row in GWAS_df.iterrows():
    if type(row['Gene_name']) == float:  #if it is null object (no gene name)
        gene_info = search_gene( row['CHROM'] , row['GENPOS'] )
        
        if gene_info != 'None':
            GWAS_df.loc[index, 'Gene_name'] = gene_info[0]
            if gene_info[1] != '':
                GWAS_df.loc[index, 'NCBI_ID'] = int(float(gene_info[1]))


#We want to add a column that will keep track of the cumulative positions of the SNPs for manhattan plot
start_pos_relative_to_genome = 0
cumulative_list = []

#groupby("CHROM") groups the CHROM column where the name of the group is the chromosome and the group is a subset of the dataframe whose chrom is of that group
for chrom, group in GWAS_df.groupby("CHROM"):
    cumulative_list.append(group["GENPOS"] + start_pos_relative_to_genome)    #store the positions of the SNPs relative to the genome, starts with 0 then the max SNP of chrom 1 and so on..
    start_pos_relative_to_genome += group["GENPOS"].max()
GWAS_df["cumulative_pos"] = pd.concat(cumulative_list)

GWAS_df["CHROM"] = GWAS_df["CHROM"].astype(str)
GWAS_df["CHROM"] = GWAS_df["CHROM"].replace({"23":"X"})

GWAS_df.to_csv('GWAS_df_updated.csv', index=False)

In [ ]:
!wsl python3 "scripts/vcf_script.py"

## <i>Now we want to study gene associations with the trait (allows us to expand on the traditional GWAS to look at genes' and bio pathways' impact on a trait)</i>

#### Setting up the GWAS summary statistics file for MAGMA (Needed for gene level association testing)

In [ ]:
GWAS_df = pd.read_csv('GWAS_df_updated.csv')
GWAS_df.drop(['EXTRA', 'TEST', 'Gene_name', 'NCBI_ID', 'color', 'def_color', 'cumulative_pos'], axis=1, inplace=True)
GWAS_df.to_csv('MAGMA_files/GWAS_summary_stat.txt', sep='\t', index = False)

#### Setting up the annotation files for MAGMA (Needed for gene level association testing)

In [ ]:
GWAS_df[['ID', 'CHROM', 'GENPOS']].to_csv('MAGMA_files/Gene-SNP_mapping_input_files/SNPLOC_file.txt', header=False, sep='\t', index=False)

#To map the genes to the SNPs
magma --annotate
      --snp-loc 'Gene-SNP_mapping_input_files/SNPLOC_file.txt'
      --gene-loc 'Gene-SNP_mapping_input_files/NCBI38.gene.loc'   #This file was downloaded from https://cncr.nl/research/magma/, gene_locations build38
      --out 'Gene-SNP_mapping_output_files/annotation_file'

#### Setting up PLINK files for MAGMA (Needed for gene level association testing)

In [ ]:
columns = ['CME', 'SNP_id', 'something', 'GENPOS', 'original_nuc', 'polymorphism']

#modify the SNP ids of the geno.bim file
geno_bim = pd.read_csv('exported_Gene_Data/PLINK_genotypes/geno.bim', sep = '\t', names = columns, header = None)

for index, row in geno_bim.iterrows():
    gwas_row_snp_id = GWAS_df.loc[index]['ID']
    
    if gwas_row_snp_id != row['SNP_id']:
        geno_bim.loc[index, 'SNP_id'] = gwas_row_snp_id
    if index == 9765:
        break
geno_bim.to_csv('MAGMA_files/MAGMA_PLINK_geno/magma_genotypes/magma_geno.bim', sep = '\t', index = False, header = None)

#Then, copy paste the other geno files (.bed and .fam) from exported_Gene_Data/PLINK_genotypes into same folder as geno.bim (magma_genotypes)

# PLINK reformats the genotype files (.bam .bed .bim) into binary format and updates them based on any changes in SNPs or sample IDs.
!wsl plink --bfile 'MAGMA_files/MAGMA_PLINK_geno/magma_genotypes/magma_geno' --make-bed --out 'MAGMA_files/MAGMA_PLINK_geno/updated_magma_genotypes/updated_magma_geno'

#### Setting up GENE-SETs.txt for gene-set level association study (Association of sets of genes with a trait)

In [ ]:
# We use reactome2py to retrieve the pathways every gene is part of and perform the classification
import reactome2py
from reactome2py import content

pathways_with_IDs_dict = {}

for index, row in GWAS_df.iterrows():
    ncbi_gene_id = row["NCBI_ID"]

    if ncbi_gene_id is not None:
        pathways = reactome2py.content.mapping(id=ncbi_gene_id, resource='NCBI gene', species='9606', by='pathways')
        if pathways is not None:
            for path in pathways: #each path is a dictionary
                path_id = path['stId']
                if path_id not in pathways_with_IDs_dict.keys():
                    pathways_with_IDs_dict[path_id] = []
                    
                if ncbi_gene_id not in pathways_with_IDs_dict[path_id]:
                    pathways_with_IDs_dict[path_id].append(ncbi_gene_id)

df = pd.DataFrame.from_dict(pathways_with_IDs_dict, orient='index')  #This makes each key as the row and its values as entries in the column cells
df = df.reset_index()  #This makes the pathway name as a separate column instead of "index"
df = df.fillna('')  #We need to replace the empty cells (represented as 'NA'/None by pandas) to avoid trailing commas when converting to txt file
df.to_csv("MAGMA_files/GENE-SETs.txt", header=False, index=False, sep='\t')

In [ ]:
#To do the gene analysis based off of their mapped SNPs(get the p-value of every gene)
#This is a prerequisite to the gene-set analysis that we are aiming for
magma --bfile 'MAGMA_PLINK_geno/updated_magma_geno'
      --pval GWAS_summary_stat.txt pval=p_values snp-id=ID ncol=N
      --gene-annot "MAGMA_files/Gene-SNP_mapping_output_files/annotation_file.genes.annot"
      --out "MAGMA_gene_analysis/gene_results"

#This is the gene-set analysis which is used to identify which sets of genes are mostly associated with the trait (larger scale than just SNPs)

#Competitive gene set analysis: Tests if genes in a set are more strongly associated with the trait than all other genes in the genome.
magma --gene-results MAGMA_gene_analysis/gene_results.genes.raw
      --set-annot GENE_SETs.txt
      --out MAGMA_gene-set_analysis/competitive_gene_set_analysis

#Self-contained get-set analysis: Tests only genes within a gene set, without comparing them to other genes.
#(is not very informative if we feed magma all the genes of the genome)
#However, if we provide a specific predefined set of genes (example: inflammation-related), then it would be very powerful
magma --gene-results MAGMA_gene_analysis/gene_results.genes.raw
      --set-annot GENE-SETs.txt
      --out MAGMA_gene-set_analysis/self-contained_analysis
      --model self-contained
      --settings gene-info

#### If GO_analysis manhattan plots are desired to be used later on, run this cell

In [ ]:
import goatools
from goatools.obo_parser import GODag
from goatools.associations import read_gaf
from goatools.base import download_go_basic_obo
from goatools.base import download_ncbi_associations
from goatools.anno.genetogo_reader import Gene2GoReader
from goatools.goea.go_enrichment_ns import GOEnrichmentStudyNS
import pronto

download_go_basic_obo("GO/")        #This retrieves all the gene ontology terms ever recorded;
                                    #it holds the definitions, relationships, and hierarchical structure of all the terms in the GO.
    
obodag = GODag("GO/go-basic.obo")   #Parses the downloaded GO ontology file (streuctures them based on relationships)
  
url = "https://ftp.ncbi.nlm.nih.gov/gene/DATA/gene2go.gz"   #The file from this link maps the GO associations with all well-studied and GO-annotated genes; needs to be downloaded then unzipped 
                                                            #I stored it as "gene2go"
    
objanno = Gene2GoReader("GO/gene2go", godag=obodag, taxids=[9606])   #We use "gene2go" (unzipped) to extract the GO mappings of only the homo sapiens genes

ontology = pronto.Ontology('GO/go-basic.obo')        #similar to obodag but it also carries the definition/description of every GO term

d = objanno.get_id2gos()   # dictionary where keys are NCBI IDs and values are GO terms

## <i>In this upcoming blocks of code, different functions have been defined to visualize GWAS results in an interacive way with the user</i>

In [ ]:
GWAS_df = pd.read_csv('GWAS_df_updated.csv')

#### function to create an interactive manhattan plot using plotly

In [ ]:
def plot(color, color_map, df=GWAS_df, highlight_color=None):
    if highlight_color==None:  #Inital plot to display
        fig = px.scatter(
        df,
        x="cumulative_pos",
        y="LOG10P",
        color= color,
        labels={"cumulative_pos": "Chromosome", "LOG10P": "-log10(p-value)"},  # Axis labels
        color_discrete_map=color_map,
        hover_data={
                    "locus":True,
                    "color":False, "cumulative_pos": False
                   }  # You can add more hover data if needed
        )
    else:
        fig = px.scatter(
        df,
        x="cumulative_pos",
        y="LOG10P",
        color= color,
        labels={"cumulative_pos": "Chromosome", "LOG10P": "-log10(p-value)"},  # Axis labels
        color_discrete_map=color_map,
        hover_data={
                    "locus":True,
                    "color":False, "cumulative_pos": False, highlight_color:False
                   }  # You can add more hover data if needed
        )
    
    # Customize x-axis ticks to show chromosomes in the middle of each block
    df["CHROM"] = df["CHROM"].replace({"X":"23"})
    df["CHROM"] = df["CHROM"].astype('int64')
    chrom_medians = df.groupby("CHROM")["cumulative_pos"].median()
    chroms = df['CHROM'].unique().tolist()
    chroms[-1] = 'X'
    fig.update_xaxes(
        tickvals=chrom_medians,  # Set the positions of ticks to the median cumulative position
        ticktext = chroms # Set the labels to chromosome names
    )
    fig.update_layout(
        plot_bgcolor='white',  # Changes the background color of the plot area
        xaxis=dict(showgrid=True,
                   gridcolor='lightblue',
                    zeroline=True,          # Show the x-axis (zeroline)
                    zerolinecolor='black',  # Set x-axis line color
                    showticklabels=True,   # Hide x-axis tick labels
                    showline=False         # Hide the axis line outside the plot),
                  ),
        yaxis=dict(showgrid=True,
                   gridcolor='lightblue',
                    zeroline=True,          # Show the x-axis (zeroline)
                    zerolinecolor='black',  # Set x-axis line color
                    showticklabels=True,   # Hide x-axis tick labels
                    showline=False)   # Hides grid lines on the y-axis
    )
    # Remove markers' outlines (similar to linewidth=0 in Seaborn)
    fig.update_traces(marker=dict(line=dict(width=0)), showlegend=False)
    
    # Show the interactive plot
    fig.show()

#### Processes a list of SNP loci to find them on the manhattan plot and highlight them for the user to see their p-value

In [ ]:
def text_box(loci_list):
         count=0
         
         for locus in loci_list:
            SNP_info = GWAS_df.loc[GWAS_df['locus'] == locus]
            
            if (pd.isna(SNP_info['Gene_name']).iloc[0] == True) and (pd.isna(SNP_info['NCBI_ID']).iloc[0] == True):  #Both are None; iloc[0] is to make sure we take only the first row of the series (it is going to be 1 anyway)
                info_text.value += f"SNP '{locus}' is not found in any known genes<br>"
                count+=1
            else:
                flag = 0
                if pd.isna(SNP_info['NCBI_ID'].iloc[0]) == False:
                    ncbi_id = SNP_info['NCBI_ID'].unique()
                    SNPs_of_gene = GWAS_df.loc[GWAS_df['NCBI_ID'].isin(ncbi_id)]
                    flag = 1
                    
                
                if pd.isna(SNP_info['Gene_name'].iloc[0]):  #if the name of the gene is unknown
                    info_text.value += f"SNP '{locus}' is found in a gene of ID: {ncbi_id[0]}<br>"
                else:
                    gene_name = SNP_info['Gene_name'].unique()
                    if flag == 1:
                        SNPs_of_gene = GWAS_df.loc[GWAS_df['Gene_name'].isin(gene_name)]
                        
                    info_text.value += f"SNP '{locus}' is found in gene {gene_name[0]}<br>"
                
                #highlight the SNP by coloring it
                gene_color = "rgb({},{},{})".format(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
                while (gene_color == 'rgb(0, 0, 255)' or gene_color == "rgb(0, 0, 0)") or gene_color == "rgb(255, 255, 255)":
                    gene_color = "rgb({},{},{})".format(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    
                GWAS_df.loc[SNPs_of_gene.index, 'color'] = gene_color
                        
                if "highlight_color" not in GWAS_df.columns:    # if it is the first SNP of the list
                            GWAS_df["highlight_color"] = GWAS_df.apply(
                                            lambda row:gene_color
                                            if row['Gene_name']==SNP_info['Gene_name'].iloc[0] else row["color"], axis=1)
                else:
                            GWAS_df["highlight_color"] = GWAS_df.apply(
                                            lambda row:gene_color
                                            if row['Gene_name']==SNP_info['Gene_name'].iloc[0] else row["highlight_color"], axis=1)
         
         if count==len(loci_list):
        #if all SNPs are not part of known genes, no need to plot figure
             pass
         else:
             with plot_output:
                        color="highlight_color"
                        color_map={"black": "black", "blue": "blue", gene_color:gene_color}
                        plot(color, color_map, GWAS_df, "highlight_color")

         GWAS_df['color'] = GWAS_df['def_color']
         GWAS_df.drop('highlight_color', axis=1, inplace=True)

def update_info(change):
    loci = change['new'].split(',')
    for locus in loci:
        locus = str(locus)
    text_box(loci)

#### Functions to display manhattan plot of related genes (user inputs in what way they are related (eg. inflammation))

In [ ]:
def GO_analysis(keyword):
    # we go over the SNPs that carry an NCBI_ID. After that, we check in d (objanno.get_id2gos()) the GO_terms of that gene.
    # If the gene is related to our keyword (eg. inflammation), we add it to the new dataframe.
    # After that, we plot this new dataframe which holds SNPs whose genes have GO-terms relating to inflammation

    SNPs_with_NCBI_IDs = GWAS_df.loc[GWAS_df['NCBI_ID'].notna()]
    gene_ids = SNPs_with_NCBI_IDs["NCBI_ID"].unique()    #stored as set
    
    SNPs_related_to_term_df = pd.DataFrame(columns = GWAS_df.columns) # an empty dataframe with same columns as GWAS_df
    
    for ID in gene_ids:
        if '|' in ID:  #some IDs have this form: 'NSBI_ID'| 'name'
            ID = ID.split('|')[0]
        ID = int(ID)
        if ID in d.keys():
            go_terms = d[ID] #go terms of gene stored as set
            for term in go_terms:
                defn = ontology[term].definition
                if keyword in defn:
                    new_row = SNPs_with_NCBI_IDs.loc[SNPs_with_NCBI_IDs['NCBI_ID'] == str(ID)]
                    SNPs_related_to_term_df = pd.concat([SNPs_related_to_term_df, new_row], ignore_index=True)
                    break  #it is enough to find only one GO-term that satisfies the condition
                    
    return SNPs_related_to_term_df

def update_GO(change):
    keyword = change['new']
    GO(keyword)

def GO(keyword):
    GO_term_df = GO_analysis(keyword)
    with plot_output:
            color= "color"
            color_map={"black": "black", "blue": "blue"}
            plot(color, color_map, GO_term_df)

#### Calls all the functions to display the manhattan plot and interactive widgets

In [ ]:
snp_text = widgets.Text(value='', placeholder='SNP locus/loci as "CME:SNP"', description='Comma-Separated SNPs:', continuous_update=False)
info_text = widgets.HTML(value='', description='Info:')
Gene_ontology = widgets.Text(value='', placeholder='Enter Gene ontology term', description='GO:', continuous_update = False)

plot_output = widgets.Output()
snp_text.observe(update_info, names="value")

#make sure to run the GO cell to prepare the dictionary d (objanno.get_id2gos())
Gene_ontology.observe(update_GO, names = "value")

display(snp_text, info_text, Gene_ontology, plot_output)

with plot_output:
    color= "color"
    color_map={"black": "black", "blue": "blue"}
    plot(color, color_map, GWAS_df)

## <i>This is where I stopped. The project was taking a machine learning approach where we use heirarchical data for feature engineering (something similar to : https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-018-2205-3 )</i>

#### Algorithm to create hierarchical tree structures of the biological pathways to better understand their relationships and p-values (not complete)

In [ ]:
import pickle
import reactome2py
from reactome2py import content

p_values = {}
with open('pathways_names_dict.pkl', 'rb') as file:  #'rb': read binary
    pathways_names_dict = pickle.load(file)
    
with open('MAGMA_files/GENE-SETs.txt', 'r') as file1, open('MAGMA_files/MAGMA_gene-set_analysis/self-contained_analysis.gsa.out', 'r') as file2:

    for _ in range(5):  #skip the first 4 lines to avoid description and header lines
        next(file2)
        
    for line1, line2 in zip(file1, file2):    #loop stops when the shorter file (file2) ends 
        nodedict = {}
        edgelist = []
        line1 = line1.strip()
        line2 = line2.strip()
        L1 = line1.split("\t")
        L2 = line2.split("      ")   #columns separated by 6 spaces
        
        path_id = L1[0]
        p_value_of_path_id = L2[-1]
        
        if path_id not in nodedict.keys():
            nodedict[path_id] = pathways_names_dict[path_id]
            
        if path_id not in p_values.keys():
            p_values[path_id] = p_value_of_path_id
            
        # list_of_pathways_ancestors = reactome2py.content.event_ancestors(id=path_id)[0]
            
        # for ancestor in list_of_pathways_ancestors:    #each ancestor is a dictionary
        #             if 'stId' in ancestor.keys():
        #                 ancestor_id = ancestor['stId']
        #                 if ancestor_id not in nodedict.keys():
        #                     nodedict[ancestor_id] = ancestor['displayName']
                            
        #                 if (ancestor_id, path_id) not in edgelist and ancestor_id != path_id:
        #                     edgelist.append( (ancestor_id, path_id) )
        #             else:
        #                 ancestor_name = ancestor['displayName']
        #                 if ancestor_name not in nodedict.keys():
        #                     nodedict[ancestor_name] = ancestor['displayName']
                            
        #                 if ancestor_name not in edgedict[path_id]:
        #                     edgelist.append((ancestor_name, path_id))
        
# with open("nodedict.pkl", "wb") as file:
#     pickle.dump(nodedict, file)
        
# with open("edgelist.pkl", "wb") as file:
#         pickle.dump(edgelist, file)
p_values

#### Load the nodes and edges created into a networkx graph (not complete)

In [ ]:
import pickle
import networkx as nx


with open(args.nodedict_file, "rb") as file:
        nodes = pickle.load(file)

with open(args.edgelist_file, "rb") as file:
        edges = pickle.load(file)
    
G = nx.DiGraph()

for path_id, path_name in nodes.items():
            G.add_node(path_id)
G.add_edges_from(edges)
connected_components = list(nx.weakly_connected_components(G))
    
for i, component in enumerate(connected_components):
    if i == 1:
        subgraph = G_pgv.subgraph(name=f"cluster_{i}")  # Create subgraph
        for node in component:
            subgraph.add_node(node)  # Add nodes
                
        edges_in_component = [edge for edge in edges if edge[0] in component and edge[1] in component]
        for edge in edges_in_component:
            subgraph.add_edge(*edge)
            